In [2]:
import requests
import pandas as pd
from datetime import date, timedelta

In [3]:
token = 'Bearer 7Q7QAeBLQYOh1ukJwp50J9u64SAM'

In [4]:
### get stock history

In [5]:
market_history = requests.get('https://sandbox.tradier.com/v1/markets/history',
    params={'symbol': 'AAPL', 'interval': 'daily', 'start': '2019-05-04', 'end': '2020-05-04'},
    headers={'Authorization': token, 'Accept': 'application/json'}
)
market_history_json_response = market_history.json()
print(market_history.status_code)

200


In [33]:
a = market_history.json()
a

{'history': {'day': [{'date': '2019-05-06',
    'open': 51.0725,
    'high': 52.21,
    'low': 50.875,
    'close': 52.12,
    'volume': 129772452},
   {'date': '2019-05-07',
    'open': 51.47,
    'high': 51.854375,
    'low': 50.20625,
    'close': 50.715,
    'volume': 155054790},
   {'date': '2019-05-08',
    'open': 50.475,
    'high': 51.335,
    'low': 50.4375,
    'close': 50.725,
    'volume': 105358016},
   {'date': '2019-05-09',
    'open': 50.1,
    'high': 50.42,
    'low': 49.165,
    'close': 50.18,
    'volume': 139634430},
   {'date': '2019-05-10',
    'open': 49.35475,
    'high': 49.7125,
    'low': 48.1925,
    'close': 49.295,
    'volume': 164834850},
   {'date': '2019-05-13',
    'open': 46.9275,
    'high': 47.37,
    'low': 45.7125,
    'close': 46.43,
    'volume': 229722490},
   {'date': '2019-05-14',
    'open': 46.6025,
    'high': 47.425,
    'low': 46.3525,
    'close': 47.165,
    'volume': 146118710},
   {'date': '2019-05-15',
    'open': 46.5675,
    '

In [25]:
### get market quotes

In [7]:
market_quotes = requests.get('https://sandbox.tradier.com/v1/markets/quotes',
    params={'symbols': 'AAPL', 'greeks': 'false'},
    headers={'Authorization': token, 'Accept': 'application/json'}
)
market_quotes_json_response = market_quotes.json()
print(market_quotes.status_code)
print(market_quotes_json_response)

200
{'quotes': {'quote': {'symbol': 'AAPL', 'description': 'Apple Inc', 'exch': 'Q', 'type': 'stock', 'last': 129.81, 'change': 2.46, 'volume': 47140613, 'open': 127.82, 'high': 129.96, 'low': 127.07, 'close': None, 'bid': 129.81, 'ask': 129.82, 'change_percentage': 1.94, 'average_volume': 89010564, 'last_volume': 300, 'trade_date': 1623691157594, 'prevclose': 127.35, 'week_52_high': 145.09, 'week_52_low': 83.145, 'bidsize': 5, 'bidexch': 'Q', 'bid_date': 1623691157000, 'asksize': 24, 'askexch': 'Q', 'ask_date': 1623691158000, 'root_symbols': 'AAPL'}}}


In [8]:
symbol = 'AAPL'
current_date = date.today()
horizon = 10 # unit days
budget = 1000 # unit dollars

print ('today is: ', current_date)

today is:  2021-06-14


In [9]:
### get option expirations

In [10]:
response = requests.get('https://sandbox.tradier.com/v1/markets/options/expirations',
    params={'symbol': symbol, 'includeAllRoots': 'true', 'strikes': 'true'},
    headers={'Authorization': token, 'Accept': 'application/json'}
)

print (response.status_code)
if response.status_code == 200:
    expiration_date_json = response.json()

200


In [11]:
def convert_expiration_date_to_df(expiration_date_json):
    length = len(expiration_date_json['expirations']['expiration'])

    ls = []
    for i in range(length):
        date = expiration_date_json['expirations']['expiration'][i]['date']
        strikes = expiration_date_json['expirations']['expiration'][i]['strikes']['strike']
        ls.append([date, strikes])

    df = pd.DataFrame(ls, columns = ['expiration_date', 'strikes'])

    return df

In [12]:
df_expiration_date = convert_expiration_date_to_df(expiration_date_json)
df_expiration_date.head()

,expiration_date,strikes
0,2021-06-18,"[18.75, 20.0, 21.25, 22.5, 23.75, 25.0, 26.25,..."
1,2021-06-25,"[65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100..."
2,2021-07-02,"[65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100..."
3,2021-07-09,"[75.0, 80.0, 85.0, 90.0, 95.0, 100.0, 105.0, 1..."
4,2021-07-16,"[22.5, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55...."


In [13]:
# Find the date satisfying conditions: expiration_date >= today + horizon

In [14]:
start_date = str(current_date + timedelta(days=horizon))
print (start_date)

df_expiration_date = df_expiration_date[df_expiration_date['expiration_date'] >= start_date]
df_expiration_date.head()

expiration_dates = df_expiration_date['expiration_date'].values

2021-06-24


In [15]:
expiration_dates

array(['2021-06-25', '2021-07-02', '2021-07-09', '2021-07-16',
       '2021-07-23', '2021-07-30', '2021-08-20', '2021-09-17',
       '2021-10-15', '2021-11-19', '2021-12-17', '2022-01-21',
       '2022-06-17', '2022-09-16', '2023-01-20', '2023-03-17',
       '2023-06-16'], dtype=object)

In [16]:
### Get option chains

In [17]:
def get_option_chains(symbol, expiration_date):
    response = requests.get('https://sandbox.tradier.com/v1/markets/options/chains',
    params={'symbol': symbol, 'expiration': expiration_date, 'greeks': 'true'},
    headers={'Authorization': token, 'Accept': 'application/json'}
    )

    if response.status_code == 200:
        json_response = response.json()
        df = pd.DataFrame(json_response['options']['option'])
    
    return df

In [18]:
expiration_dates[0]

'2021-06-25'

In [19]:
df_option_chains = get_option_chains(symbol, expiration_dates[0])
df_option_chains.head()

,symbol,description,exch,type,last,change,volume,open,high,low,...,bid_date,asksize,askexch,ask_date,open_interest,contract_size,expiration_date,expiration_type,option_type,root_symbol
0,AAPL210625P00065000,AAPL Jun 25 2021 $65.00 Put,Z,option,0.02,0.0,0,NaN,NaN,NaN,...,1623690681000,10,Z,1623690645000,299,100,2021-06-25,weeklys,put,AAPL
1,AAPL210625C00065000,AAPL Jun 25 2021 $65.00 Call,Z,option,60.60,0.0,0,NaN,NaN,NaN,...,1623691163000,10,X,1623691162000,1,100,2021-06-25,weeklys,call,AAPL
2,AAPL210625P00070000,AAPL Jun 25 2021 $70.00 Put,Z,option,0.02,0.0,0,NaN,NaN,NaN,...,1623690681000,639,C,1623691160000,25,100,2021-06-25,weeklys,put,AAPL
3,AAPL210625C00070000,AAPL Jun 25 2021 $70.00 Call,Z,option,NaN,NaN,0,NaN,NaN,NaN,...,1623691163000,124,C,1623691163000,0,100,2021-06-25,weeklys,call,AAPL
4,AAPL210625P00075000,AAPL Jun 25 2021 $75.00 Put,Z,option,0.01,0.0,0,NaN,NaN,NaN,...,1623690465000,5,Z,1623685924000,26,100,2021-06-25,weeklys,put,AAPL


In [20]:
df_option_chains_filtered = df_option_chains[(df_option_chains['ask'] * 100 >= 0.9*budget) & (df_option_chains['ask'] * 100 <= budget) & (df_option_chains['option_type'] == 'call')]

In [21]:
df_option_chains_filtered.head()

,symbol,description,exch,type,last,change,volume,open,high,low,...,bid_date,asksize,askexch,ask_date,open_interest,contract_size,expiration_date,expiration_type,option_type,root_symbol
43,AAPL210625C00121000,AAPL Jun 25 2021 $121.00 Call,Z,option,8.75,2.23,62,6.7,8.82,6.7,...,1623691165000,137,C,1623691165000,593,100,2021-06-25,weeklys,call,AAPL


In [22]:
import requests

In [23]:
url = "https://tradier.p.rapidapi.com/markets/quote"

querystring = {"symbols":"aapl,goog,ibm"}

headers = {
    'authorization': token,
    'x-rapidapi-key': "7Q7QAeBLQYOh1ukJwp50J9u64SAM",
    'x-rapidapi-host': "tradier.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message":"You are not subscribed to this API."}


In [24]:
def get_option_chains(symbol, expiration_date):
    response = requests.get('https://sandbox.tradier.com/v1/markets/options/chains',
    params={'symbol': symbol, 'expiration': expiration_date, 'greeks': 'true'},
    headers={'Authorization': token, 'Accept': 'application/json'}
    )

    if response.status_code == 200:
        json_response = response.json()
        df = pd.DataFrame(json_response['options']['option'])
    
    return df


symbol = "SPX"
expiration_dates = '2021-07-06'

df_option_chains = get_option_chains(symbol, expiration_dates)
df_option_chains.head()

,symbol,description,exch,type,last,change,volume,open,high,low,...,bid_date,asksize,askexch,ask_date,open_interest,contract_size,expiration_date,expiration_type,option_type,root_symbol
0,SPXW210706P01200000,SPXW Jul 6 2021 $1200.00 Put,C,option,NaN,NaN,0,NaN,NaN,NaN,...,1622790000000,232,C,1623692043000,0,100,2021-07-06,weeklys,put,SPXW
1,SPXW210706C01200000,SPXW Jul 6 2021 $1200.00 Call,C,option,NaN,NaN,0,NaN,NaN,NaN,...,1623692064000,30,C,1623692064000,0,100,2021-07-06,weeklys,call,SPXW
2,SPXW210706P01400000,SPXW Jul 6 2021 $1400.00 Put,C,option,NaN,NaN,0,NaN,NaN,NaN,...,1622790000000,231,C,1623692043000,0,100,2021-07-06,weeklys,put,SPXW
3,SPXW210706C01400000,SPXW Jul 6 2021 $1400.00 Call,C,option,NaN,NaN,0,NaN,NaN,NaN,...,1623692070000,30,C,1623692058000,0,100,2021-07-06,weeklys,call,SPXW
4,SPXW210706C01600000,SPXW Jul 6 2021 $1600.00 Call,C,option,NaN,NaN,0,NaN,NaN,NaN,...,1623692064000,30,C,1623692064000,0,100,2021-07-06,weeklys,call,SPXW
